**DEATH PREDICT**

*Predicting heart failure in hospital ICU (Intensive car Unit)*

In [110]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [111]:
# Read CSV file into DataFrame df
df = pd.read_csv('/kaggle/input/in-hospital-mortality-prediction/data01.csv', index_col=0)
# Show dataframe
df

In [112]:
df.describe()

In [113]:
df.info()

In [114]:
df.dropna(inplace=True)
df.info

In [115]:
# Average Age of non-surviving patients 
deads = df[df.outcome == 1]
print(np.mean(deads.age))

In [116]:
import seaborn as sns
corr = df.corr()
sns.heatmap(corr,
           xticklabels=corr.columns.values,
           yticklabels=corr.columns.values)

In [117]:
# Visualization of Heart Failures
'''
fig, ax = plt.subplots()
dead_count = df['outcome'].value_counts() 
dead_count.plot(ax=ax, kind='bar')
'''

In [118]:
# Visualization of Heart Failures

plt.figure(figsize=(16,8))
plt.title('Frequency of Heart Failures')
plt.hist(df['outcome'])
plt.xlabel('Heart Failure (0 = No) (1 = Yes)', fontsize=18)
plt.ylabel('Number of patients', fontsize=18)
plt.show()

**we can see that 13% of all patients died of heart failure during the study, which is not evenly balanced**

In [119]:
# create the labels (ground truth) and features (data to train)
labels = df['outcome']

# Eliminate Nan values
# replace Nan by the mean of the column
features = df.iloc[:, 2:]
for i in range(features.shape[1]):
    features.update(features.iloc[:,i].fillna(value=features.iloc[:,i].mean(), inplace=True))

X = features

y = np.ravel(labels)

In [120]:
# creating train and test data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.33, random_state = 42)

**DECISION TREE**

In [121]:
from sklearn.tree import DecisionTreeClassifier

#instanciation of the decision tree
model = DecisionTreeClassifier()

In [122]:
#train the model
model.fit(X_train, y_train)

In [123]:
# get the models predicted outcome (death)
y_predictions = (model.predict(X_test) > 0.5).astype("int32")

In [124]:
score = model.score(X_test, y_test)
score

THE DECISION TREE GIVES A BETTER PREDICTION THAN OUR INITIAL NEURAL NETWORK : ~80% instead of ~40%